# a. BASIC RULE-BASED CHATBOT USING PYTHON NLTK 

In [11]:
import nltk
from nltk.chat.util import Chat, reflections
pairs = [
    (r"my name is (.*)", ["Hello %1, how are you today?"]),
    (r"hi|hey|hello", ["Hello!", "Hey there!"]),
    (r"what is your name?", ["I am a bot created by [Your Name]."]),
    (r"how are you?", ["I'm doing good. How about you?"]),
    (r"sorry (.*)", ["No problem!", "It's okay.", "You don't need to be sorry."]),
    (r"quit", ["Bye! Take care."]),
]
def chatbot():
    print("Hi, I'm the chatbot you created. Type 'quit' to exit.")
    chat = Chat(pairs, reflections)
    chat.converse()
if __name__ == "__main__":
    chatbot()

Hi, I'm the chatbot you created. Type 'quit' to exit.


> my name is nithish


Hello nithish, how are you today?


> quit


Bye! Take care.


# b.  BUILDING A CHATBOT USING SEQ2SEQ MODELS(google collab)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# --- Toy dataset ---
inputs = ["Hi", "who are you?"]
targets = ["Hello!", "I'm good!"]
chars = sorted(list(set(" ".join(inputs + targets))))
c2i = {c:i for i,c in enumerate(chars)}
i2c = {i:c for c,i in c2i.items()}
max_in, max_out = max(len(s) for s in inputs), max(len(s) for s in targets)

def encode(s, l): return [c2i[c] for c in s] + [0]*(l-len(s))
X_enc = np.array([encode(s, max_in) for s in inputs])
X_dec = np.array([encode(s, max_out) for s in targets])
Y = np.expand_dims(X_dec, -1)

# --- Seq2Seq Model ---
latent=32
enc_inp = Input((max_in,))
enc_emb = Embedding(len(chars), latent)(enc_inp)
enc_out, h, c = LSTM(latent, return_state=True)(enc_emb)

dec_inp = Input((max_out,))
dec_emb = Embedding(len(chars), latent)(dec_inp)
dec_out, _, _ = LSTM(latent, return_sequences=True, return_state=True)(dec_emb, initial_state=[h, c])
dec_out = Dense(len(chars), activation='softmax')(dec_out)

model = Model([enc_inp, dec_inp], dec_out)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit([X_enc, X_dec], Y, epochs=300, verbose=0)

# --- Inference ---
def respond(s):
    seq = np.array([encode(s, max_in)])
    dec = np.zeros((1, max_out))
    for i in range(max_out):
        p = model.predict([seq, dec], verbose=0)[0]
        dec[0,i] = np.argmax(p[i])
    return "".join([i2c[int(i)] for i in dec[0]]).strip()

# Test
print("Input: How are you?")
print("Bot:", respond("How are you?"))


# C. CONVERSATIONAL AI WITH TRANSFORMER-BASED MODELS(Google collab)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
chat_history_ids = None
print(" Conversational AI Chatbot (type 'quit' to exit)")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Bot: Goodbye! 👋")
        break
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        bot_input_ids = new_input_ids
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    bot_output = tokenizer.decode(
        chat_history_ids[:, bot_input_ids.shape[-1]:][0],
        skip_special_tokens=True
    )
    print("Bot:", bot_output)

OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.